In [1]:
from functions import *
from Twitter_functions import *
import json

In [2]:
with open('Authentication/twitter_bearer_token.txt', 'r', encoding="utf8") as f:
    token = f.read()

header = create_Twitter_headers(token)

with open('Authentication/database_uri.txt', 'r', encoding="utf8") as f:
    uri = f.read()

In [3]:
def checkForRepliesToNews(tweetDict):
    result = []
    for i in tweetDict['data']:
        reply = re.findall("(@[A-Za-z0-9]+)", i['text'])

        if any(x in ['@MothershipSG', '@straits_times', '@ChannelNewsAsia', '@YahooSG'] for x in reply):
            result.append(i)

    return result

def getAndSaveReplyNRetweets(users, filterForNewsProvider, noUserId = True):
    result = {}
    
    for i in users:
        
        if noUserId:
            userID = getTwitterUserInfo(i, header)['data'][0]['id']
            tweets = getTweetsByUserID(userID, header)
        
        else:
            tweets = getTweetsLikedByUser(i, header)
        
        if filterForNewsProvider:
            filterTweets = checkForRepliesToNews(tweets)
            
            if len(filterTweets) > 0:
                
                if noUserId:
                    result[userID] = filterTweets
                    
                else:
                    result[i] = filterTweets
        
        else:
            if noUserId:
                result[userID] = tweets['data']
                
            else:
                result[i] = tweets['data']
        
    return result

def getAndSaveLikedTweets(users, filterForNewsProvider, noUserId = True):
    result = {}
    
    for i in users:
        
        if noUserId:
            userID = getTwitterUserInfo(i, header)['data'][0]['id']
            tweets = getTweetsLikedByUser(userID, header)
            
        else:
            tweets = getTweetsLikedByUser(i, header)
        
        if filterForNewsProvider:
            filterTweets = checkForRepliesToNews(tweets)
            
            if len(filterTweets) > 0:
                
                if noUserId:
                    result[userID] = filterTweets
                    
                else:
                    result[i] = filterTweets
        
        else:
            if noUserId:
                result[userID] = tweets['data']
            
            else:
                result[i] = tweets['data']
        
    return result

def getCommentsFromDBAndSave(conversationID, dbName, haveUserId = True):
    result = {'data':[]}
    
    command = "select * from %s where conversation_id = '%s';" % (dbName, conversationID)
    conn = psycopg2.connect(uri, sslmode='require')

    try:
        cur = conn.cursor()
        cur.execute(command)
        row = cur.fetchall()

        for i in row:
            resultDict = {}
            resultDict['id'] = i[0]
            resultDict['timestamp'] = i[1]
            resultDict['reply_to'] = i[2]
            resultDict['comment'] = i[3]
            resultDict['social_media'] = i[4]
            resultDict['head_id'] = i[5]
            resultDict['conversation_id'] = i[6]
            
            if haveUserId:
                resultDict['user_id'] = i[7]
            
            result['data'].append(resultDict)

    except (Exception, psycopg2.DatabaseError) as error:
        return error
    
    return result

def getCommentsFromDBAndSave_PHEME(conversationID, dbName, haveUserId = True):
    result = {'data':[]}
    
    command = "select * from %s where head_id = '%s';" % (dbName, conversationID)
    conn = psycopg2.connect(uri, sslmode='require')

    try:
        cur = conn.cursor()
        cur.execute(command)
        row = cur.fetchall()

        for i in row:
            resultDict = {}
            resultDict['id'] = i[0]
            resultDict['timestamp'] = i[1]
            resultDict['reply_to'] = i[2]
            resultDict['comment'] = i[3]
            resultDict['head_id'] = i[4]
            result['data'].append(resultDict)

    except (Exception, psycopg2.DatabaseError) as error:
        return error
    
    return result

# Case Study 1

In [4]:
cs1_comments = getCommentsFromDBAndSave_PHEME('553553331671408641', 'pheme_dataset_for_analysis', False)

with open('Collected Data/case_study_1_all_comments.json', 'w') as fp:
    json.dump(cs1_comments, fp)

# Case Study 2

In [5]:
# cs2_comments = getCommentsFromDBAndSave('1507922082683793408', 'comments_for_analysis', False)

# with open('Collected Data/case_study_2_all_comments.json', 'w') as fp:
#     json.dump(cs2_comments, fp)

In [6]:
# suspectedBiasUsers = ['CardanoAdonis', 'PLafala', 'Erwin_Dawson', 'Coldcappuccino9', '_5andman_', 'Actarus_dEuphor', 'ResenT___', 'mkggoh', 'slowpokemax']

# cs2_dataDict_1 = getAndSaveReplyNRetweets(suspectedBiasUsers, False)

# with open('Collected Data/case_study_2_all_bias_users_retweet_replies.json', 'w') as fp:
#     json.dump(cs2_dataDict_1, fp)

In [7]:
# cs2_dataDict_2 = getAndSaveReplyNRetweets(suspectedBiasUsers, True)

# with open('Collected Data/case_study_2_all_bias_users_retweet_replies_news_providers_filter.json', 'w') as fp:
#     json.dump(cs2_dataDict_2, fp)

In [8]:
# cs2_dataDict_1_likes = getAndSaveLikedTweets(suspectedBiasUsers, False)

# with open('Collected Data/case_study_2_all_potentially_bias_users_liked_tweets.json', 'w') as fp:
#     json.dump(cs2_dataDict_1_likes, fp)

In [9]:
# cs2_dataDict_2_likes = getAndSaveLikedTweets(suspectedBiasUsers, True)

# with open('Collected Data/case_study_2_all_potentially_bias_users_liked_tweets_news_providers_filter.json', 'w') as fp:
#     json.dump(cs2_dataDict_2_likes, fp)

# Case Study 3

In [10]:
# cs3_comments = getCommentsFromDBAndSave('1522931750451617793', 'comments_for_analysis', True)

# with open('Collected Data/case_study_3_all_comments.json', 'w') as fp:
#     json.dump(cs3_comments, fp)

In [11]:
# suspectedBiasUsers = ['1531214377', '804973086567055360', '83822931', '3246610436', '1365682034551627779', 
#                       '1339037189565415425', '1499140477718872072', '1266607549245091840', '890795094256648192',
#                       '1509197192459816962', '1422221555950518275']

# cs3_dataDict_1 = getAndSaveReplyNRetweets(suspectedBiasUsers, False, False)

# with open('Collected Data/case_study_3_all_bias_users_retweet_replies.json', 'w') as fp:
#     json.dump(cs3_dataDict_1, fp)

In [12]:
# cs3_dataDict_2 = getAndSaveReplyNRetweets(suspectedBiasUsers, True, False)

# with open('Collected Data/case_study_3_all_bias_users_retweet_replies_news_providers_filter.json', 'w') as fp:
#     json.dump(cs3_dataDict_2, fp)

In [13]:
# cs3_dataDict_1_likes = getAndSaveLikedTweets(suspectedBiasUsers, False, False)

# with open('Collected Data/case_study_3_all_potentially_bias_users_liked_tweets.json', 'w') as fp:
#     json.dump(cs3_dataDict_1_likes, fp)

In [14]:
# cs2_dataDict_3_likes = getAndSaveLikedTweets(suspectedBiasUsers, True, False)

# with open('Collected Data/case_study_3_all_potentially_bias_users_liked_tweets_news_providers_filter.json', 'w') as fp:
#     json.dump(cs2_dataDict_3_likes, fp)